In [8]:
cd ../../../../

/


/Users/mykhailoslukvin/.local/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [9]:
import asyncio
from tqdm.asyncio import tqdm_asyncio
import pandas as pd
from pprint import pprint as print

from dfpp.geo_utils import get_iso3_to_official_name_map
from dfpp.transformation.source_notebooks.who_azureedge_net.retrieve import (
    list_indicators,
    get_indicator_data,
    get_all_dimension_values,
    list_dimensions,
)
from dfpp.transformation.source_notebooks.who_azureedge_net.transform import (
    process_dimension_data, transform_indicator, SOURCE_NAME
)
from dfpp.publishing.publish import publish_series

MAX_CONCURRENCY = 1

In [10]:
ISO_3_MAP = await get_iso3_to_official_name_map()

In [11]:
df_indicators = pd.DataFrame(list_indicators())

In [12]:
df_dimension_codes = pd.DataFrame(list_dimensions())

In [13]:
df_dimension_values = await get_all_dimension_values(df_dimension_codes)

In [14]:
df_full_dimension_map = df_dimension_codes.merge(
    df_dimension_values,
    left_on="Code",
    right_on="Dimension",
    how="left",
    suffixes=("_dimension", "_value"),
)

In [15]:
df_full_dimension_map = process_dimension_data(df_full_dimension_map)

In [16]:
async def process_indicator(indicator_code: str, semaphore: asyncio.Semaphore, df_full_dimension_map: pd.DataFrame):
    try:
        async with semaphore:
            indicator_data = await get_indicator_data(indicator_code)
            df_source = pd.DataFrame(indicator_data["value"])

            if df_source.empty:
                return None
            df = df_source.copy()
            df = transform_indicator(indicator_code, df, df_full_dimension_map, ISO_3_MAP)
            if df.empty:
                return None
            await publish_series(indicator_code, df, source_folder=SOURCE_NAME)
    except Exception as e:
        return indicator_code, e

In [17]:
async def process_all_indicators(
    df_indicators: pd.DataFrame,
    df_full_dimension_map: pd.DataFrame,
    max_concurrency: int = MAX_CONCURRENCY,
):
    semaphore = asyncio.Semaphore(max_concurrency)

    failed_indicators = []

    tasks = [
        process_indicator(indicator_code, semaphore, df_full_dimension_map)
        for indicator_code in df_indicators.IndicatorCode.tolist()
    ]

    for future in tqdm_asyncio.as_completed(tasks):
        result = await future
        if isinstance(result, tuple) and len(result) == 2:
            print(result)
            failed_indicators.append(result)
    return failed_indicators

In [18]:
failed_indicators = await process_all_indicators(
    df_indicators, df_full_dimension_map
)

  0%|          | 0/2914 [00:00<?, ?it/s]

  0%|          | 4/2914 [00:18<4:59:25,  6.17s/it]

('W_Group',
 ClientConnectorError(ConnectionKey(host='ghoapi.azureedge.net', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=None), TimeoutError(60, "Connect call failed ('152.199.19.161', 443)")))


  0%|          | 13/2914 [00:33<1:18:50,  1.63s/it]

('OCC_12', AttributeError("'NoneType' object has no attribute 'empty'"))


...

('OCC_19',
 ClientConnectorError(ConnectionKey(host='ghoapi.azureedge.net', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=None), TimeoutError(60, "Connect call failed ('152.199.19.161', 443)")))


  2%|▏         | 44/2914 [02:25<1:48:17,  2.26s/it]

('OCC_2', AttributeError("'NoneType' object has no attribute 'empty'"))


  2%|▏         | 47/2914 [02:33<2:35:48,  3.26s/it]


CancelledError: 

In [12]:
assert len(failed_indicators) == 0, print(failed_indicators)